In [1]:
import pandas as pd
import numpy as np
from numba import njit, prange

from tqdm.notebook import tqdm, trange

from numba.typed import List
from numba import types

from numba_progress import ProgressBar

import math

In [2]:
users = 226570
items = 231637

train_data = pd.read_csv("../Preprocessing/processed_dataframes/train.csv")
validation_data = pd.read_csv("../Preprocessing/processed_dataframes/val.csv")

In [3]:
uir_train = train_data.values

uir_val = validation_data.values
n_val = uir_val.shape[0]

In [4]:
@njit
def step(
    train_data, 
    n_users, 
    n_items, 
    k, 
    α1,
    α2,
    α3,
    α4,
    λ1, 
    λ2,
    μ, bi, bu, P, Q,
):
    loss = 0
    for u, i, r in train_data:
        pred = μ + bu[u] + bi[i] + np.dot(Q[i], P[u])
        error = r - pred

        # Updating
        bu[u] += α1 * (error - λ1*bu[u])
        bi[i] += α2 * (error - λ1*bi[i])

        Pu = P[u]
        Qi = Q[i]
        P[u] += α3*(error*Qi - λ2*Pu)
        Q[i] += α4*(error*Pu - λ2*Qi)
            
        loss += error**2
            
    return np.sqrt(loss/len(train_data))

In [5]:
# RS HD page 171 (chrome), 84 book
def fit_svd(
    train_data, val_data, n_users, n_items, k, α1=.01, α2=.01, α3=.01, α4=.01, λ1=.01, λ2=.01, n_iters=20
):
    """
    train_data: array Nx3
    """
    val_ui = uir_val[:, :2]
    val_exp = uir_val[:, -1]
    
    bu = np.zeros(n_users, np.double)
    bi = np.zeros(n_items, np.double)
    
    P = np.random.normal(0, .1, (n_users, k))
    Q = np.random.normal(0, .1, (n_items, k))
    
    μ = np.mean(train_data[:, 2])
    
    model_params = None
    best_epoch = 0
    prev_val_loss = math.inf
    
    t = trange(n_iters, leave=True)
    for it in t:
#     for it in range(n_iters):
        loss = step(train_data, n_users, n_items, k, α1, α2, α3, α4, λ1, λ2, μ, bi, bu, P, Q)
        
        val_preds = predict_batch(val_ui, (μ, bu, bi, P, Q))
        val_loss = np.sqrt(1/n_val * np.sum((val_preds - val_exp)**2))
        t.set_postfix({"Loss": loss, "Val": val_loss})
        
        if val_loss < prev_val_loss:
            prev_val_loss = val_loss
            model_params = (μ, bu.copy(), bi.copy(), P.copy(), Q.copy())
            best_epoch = it
    
#     return μ, bu, bi, P, Q
    return model_params

## Fit

In [6]:
@njit
def predict(u, i, params):
    μ, bu, bi, P, Q = params
    k = P.shape[1]

    pred = μ + bu[u] + bi[i] + np.dot(Q[i], P[u])
    
    return pred

In [7]:
@njit(parallel=True, nogil=True)
def predict_batch(ui_mat, params):
    predictions = np.zeros(len(ui_mat))
    for it in prange(ui_mat.shape[0]):
        u, i = ui_mat[it]
        predictions[it] = predict(u, i, params)
        
    return np.clip(predictions, 1., 5.)

In [8]:
α1 = 0.005
α2 = 0.005
α3 = 0.01
α4 = 0.01
lamb_1 = 0.05
lamb_2 = 0.1
# 4, 5, 6, 50, 100, 150
k = 150
fitted_params = fit_svd(
    uir_train, uir_val, users, items, k, 
    α1, α2, α3, α4, λ1=lamb_1, λ2=lamb_2,
    n_iters=75,
)

  0%|          | 0/75 [00:00<?, ?it/s]

In [11]:
val_preds = predict_batch(uir_val[:, :2], fitted_params)
val_expected = uir_val[:, 2]

error = np.sqrt(1/n_val * np.sum((val_preds - val_expected)**2))
print(error)

0.9035591274599666


## Multiple train

In [12]:
from joblib import Parallel, delayed
from itertools import product

In [13]:
alphas = [0.005, 0.006, 0.007, 0.01]
lambdas = [0.008, 0.01, 0.05, 0.1]
factors = [4, 5, 6, 50, 100, 150]

params_product = list(product(alphas, alphas, lambdas, lambdas, factors))

In [14]:
def train_val(
    uir_train,
    uir_val,
    users,
    movies,
    k,
    α1,
    α2,
    α3,
    α4,
    λ1,
    λ2,
    n_iters,
):
    fitted_params = fit_svd(uir_train, uir_val, users, movies, k, α1, α2, α3, α4, λ1, λ2, n_iters)
    
    val_preds = predict_batch(uir_val[:, :2], fitted_params)
    val_expected = uir_val[:, 2]
    error = np.sqrt(1/n_val * np.sum((val_preds - val_expected)**2))
    
    return α1, α2, α3, α4, λ1, λ2, k, error

In [16]:
out = Parallel(n_jobs=12)(
    delayed(train_val)(
        uir_train, 
        uir_val,
        users, 
        items,
        k=f, 
        α1=lr1,
        α2=lr1,
        α3=lr2,
        α4=lr2,
        λ1=lamb1, 
        λ2=lamb2, 
        n_iters=100,
    )
    for lr1, lr2, lamb1, lamb2, f in tqdm(params_product)
)

  0%|          | 0/1536 [00:00<?, ?it/s]

In [32]:
best_params_joblib = sorted(out, key=lambda x: x[-1])[0]

print("α1: {0}, α2: {1}, α3: {2}, α4: {3}, λ1: {4}, λ2: {5}, k: {6}, error: {7}".format(*best_params_joblib))

α1: 0.005, α2: 0.005, α3: 0.005, α4: 0.005, λ1: 0.01, λ2: 0.1, k: 4, error: 0.8986038030954508
